In [2]:
import math
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [15, 10]

In [3]:
from rpy2.robjects.packages import importr

utils = importr("utils")
utils.chooseCRANmirror(ind=1)
#utils.install_packages('lme4')

%load_ext rpy2.ipython

R[write to console]: Warning:
R[write to console]:  failed to download mirrors file (cannot open URL 'https://cran.r-project.org/CRAN_mirrors.csv'); using local file '/home/mitja/anaconda3/envs/cf_timing/lib/R/doc/CRAN_mirrors.csv'



In [4]:
# Load data
conversations = pd.read_csv("results/reproduce_warlaumont/conversations.csv")

# convert True/False to 0/1:
conversations.replace({False: 0, True: 1}, inplace=True)

# normalize age
min_age, max_age = conversations["age"].min(), conversations["age"].max()
conversations["age"] = (conversations["age"] - min_age) / (max_age - min_age) * (1 - 0)

conversations.head()

,utterance_id,speaker_code,tokens,pos,age,corpus,transcript_file,child_name,speaker_code_next,start_time_next,...,response_speech_act,follow_up_transcript_raw,follow_up_start_time,follow_up_end_time,follow_up_is_speech_related,follow_up_is_intelligible,follow_up_speech_act,response_latency,response_latency_follow_up,has_response
0,2004,CHI,['.'],['none'],0.289474,Bloom,/home/mitja/data/CHILDES/Bloom/Peter/010908.cha,Bloom_Peter,MOT,3534418.0,...,QN,frisbee .,3550229.0,3552151.0,1,1,SA,0.0,15811.0,1
1,2009,CHI,"['frisbee', '.']",['n'],0.289474,Bloom,/home/mitja/data/CHILDES/Bloom/Peter/010908.cha,Bloom_Peter,MOT,3552151.0,...,AP,&=laughs .,3560697.0,3561813.0,0,0,YY,0.0,8546.0,1
2,2025,CHI,"['train', 'train', 'penny', '.']","['n', 'v', 'n']",0.289474,Bloom,/home/mitja/data/CHILDES/Bloom/Peter/010908.cha,Bloom_Peter,MOT,3625437.0,...,ST,all gone .,3630262.0,3632581.0,1,1,DC,0.0,4825.0,1
3,2043,CHI,"['car_car', '.']",['chi'],0.289474,Bloom,/home/mitja/data/CHILDES/Bloom/Peter/010908.cha,Bloom_Peter,MOT,3710723.0,...,QN,no .,3712392.0,3714645.0,1,1,AN,0.0,1669.0,1
4,2046,CHI,"['no', '.']",['co'],0.289474,Bloom,/home/mitja/data/CHILDES/Bloom/Peter/010908.cha,Bloom_Peter,MOT,3713487.0,...,QN,mm .,3715173.0,3717109.0,1,1,SA,-1158.0,528.0,1


## Quality of communicative feedback/ Caregiver contingency


### Timing:

In [5]:
%%R -i conversations
library(lme4)

m_caregiver_contingency<-glmer('has_response ~ utt_is_speech_related * age + (1 | child_name)', data=conversations, family=binomial)
print(summary(m_caregiver_contingency))


R[write to console]: Loading required package: Matrix



Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: has_response ~ utt_is_speech_related * age + (1 | child_name)
   Data: conversations

      AIC       BIC    logLik  deviance  df.resid 
 345448.5  345503.1 -172719.2  345438.5    404897 

Scaled residuals: 
     Min       1Q   Median       3Q      Max 
-19.7761   0.0802   0.4767   0.5529   1.4055 

Random effects:
 Groups     Name        Variance Std.Dev.
 child_name (Intercept) 3.146    1.774   
Number of obs: 404902, groups:  child_name, 179

Fixed effects:
                          Estimate Std. Error z value Pr(>|z|)    
(Intercept)                3.36854    0.08564  39.333  < 2e-16 ***
utt_is_speech_related      0.10328    0.02984   3.462 0.000537 ***
age                        0.27554    0.05168   5.332 9.74e-08 ***
utt_is_speech_related:age  0.96321    0.05060  19.037  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1



## Effect of communicative feedback
### Positive Feedback: Timing

In [8]:
%%R -i conversations
library(lme4)

conversations_child_speech_related = subset(conversations, utt_is_speech_related==1)

m_child_contingency<-glmer('follow_up_is_speech_related ~ has_response * age + (1 | child_name)', data=conversations_child_speech_related, family=binomial)
print(summary(m_child_contingency))

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: follow_up_is_speech_related ~ has_response * age + (1 | child_name)
   Data: conversations_child_speech_related

     AIC      BIC   logLik deviance df.resid 
 83541.5  83595.9 -41765.8  83531.5   391165 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-45.127   0.063   0.157   0.168   0.909 

Random effects:
 Groups     Name        Variance Std.Dev.
 child_name (Intercept) 2.86     1.691   
Number of obs: 391170, groups:  child_name, 179

Fixed effects:
                 Estimate Std. Error z value Pr(>|z|)    
(Intercept)       4.14678    0.12491  33.199  < 2e-16 ***
has_response     -0.04065    0.05180  -0.785    0.433    
age              -0.07379    0.09805  -0.753    0.452    
has_response:age  0.39662    0.09454   4.195 2.73e-05 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Correlation of Fixed Effects:
        